# Scrapowanie Airbnb przy użyciu API Scraping Fish

Ten notebook to przykład jak można scrapować stronę renderującą swoją treść za pomocą JavaScriptu, korzystając z [API Scraping Fish](https://scrapingfish.pl). Będziemy pobierać dane ofert z Airbnb i dokonamy podstawowej eksploracji danych. Notebook ten jest związany z [postem na blogu](https://scrapingfish.pl/blog/scraping-airbnb), który stanowi dokładniejszy opis tego kodu.

Żeby móc uruchomić ten notebook i faktycznie pobrać dane, będziesz potrzebować klucza API Scraping Fish, który możesz [kupić tutaj](https://scrapingfish.pl/buy). Podstawowa paczka requestów kosztuje 8zł + VAT i w zupełności wystarczy, żeby uruchomić ten notebook wiele razy. Bez tego, JavaScript nie wyrenderuje zawartości i nie będzie można jej pobrać ze strony.

Scraping Fish to api do scrapowania oparte o rotujące mobilne proxy 4G/LTE. To najlepszy typ proxy do scrapowania ponieważ mobilne adresy IP są ulotne i często zmieniają się pomiędzy prawdziwmy użytkownikami. Ten typ proxy jest w stanie zapewnić nieprzerwane scrapowanie nawet dla najbardziej wymagających stron bez bycia zablokowanym. Możesz przeczytać więcej o API Scraping Fish w [Dokumentacji](https://scrapingfish.pl/docs/intro).

## Importy
Potrzebne pakiety, które importujemy poniżej są wylistowane w pliku `requirements.txt`. Przed uruchomieniem notebooka zainstaluj je: `pip install -r requirements.txt`.

In [ ]:
import requests
import pandas as pd
import datetime
import seaborn as sns
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from tqdm import tqdm
import matplotlib.pyplot as plt

sns.set_style("darkgrid")
sns.set(font_scale=1.3)

## Klucz API
Klucz API Scraping Fish jest wymagany do uruchomienia tego przykładu dlatego, że dzięki temu będziemy mogli renderować JavaScript. Jedyne co jest wymagane do integracji to dodanie poniższego prefixu do każdego scrapowanego urla.

Możesz [zdobyć klucz API](https://scrapingfish.pl/buy) kupując najtańszą paczkę za 8 zł + VAT.

In [ ]:
API_KEY = "[Your Scraping Fish API key]"
url_prefix = f"https://scraping.narf.ai/api/v1/?render_js=true&api_key={API_KEY}&url="

## Parametry
Poniżej definiujemy parametry dla naszego scrapowania. W tym przykładzie pobieramy oferty z Warszawy dostępne między 1 czerwca 2022 a 14 października 2022, pobierając z częstotliwością 45 dni. Możesz zmienić te parametry na jakiekolwiek chcesz, ale miej na uwadze, że zmiana częstotliwości (`frequency`) do małej wartości (np. `"1D"`) znacznie wydłuży czas scrapowania i niekoniecznie zwiększy ilość zebranych danych.

In [ ]:
query = "Warszawa"
start_date = datetime.date(2022, 6, 1)
end_date = datetime.date(2022, 10, 14)
freq = '45D'
num_guests = 1

## Scrapowanie i Parsowanie
Dla danego zapytania Airbnb listuje 300 ofert (15 stron, 20 ofert na każdej stronie). Będziemy scrapować wszystkie 15 stron dla każdego dnia, wyciągać informacje z każdej z ofert i dodawać je do kolekcji.

### Parsowanie pojedynczej oferty
Najpierw zdefiniujemy funkcję odpowiedzialną za parsowanie pojedycznej oferty. Będzie ona wywoływana dla każdej oferty na stronie żeby wydobyć interesujące nas informacje i zapisać je do słownika, który potem będzie dodany do kolekcji wszystkich ofert. Musimy też zdefiniować mapowanie kluczy, żeby ustandaryzować m.in. liczbę mnogą i pojedynczą do wspólnego klucza.

BeatufiulSoup jest tu używany do wybierania pod-elementów z elementu głównego zawierającego informacje o ofercie.

In [ ]:
key_map = {
    'łazienka': 'bathrooms',
    'łazienki': 'bathrooms',
    'łazienek': 'bathrooms',
    'łazienki': 'bathrooms',
    'łazieniek': 'bathrooms', # yup, a typo on Airbnb's website
    'współdzielona łazienka': 'bathrooms',
    'współdzielone łazienki': 'bathrooms',
    'współdzielonych łazienek': 'bathrooms',
    'wspólnej łazienki': 'bathrooms',
    'wspólnych łazienek': 'bathrooms',
    'prywatna łazienka': 'bathrooms',
    'prywatne łazienki': 'bathrooms',
    'prywatnych łazienek': 'bathrooms',
    'toaleta': 'bathrooms',
    'toalet': 'bathrooms',
    'toalety': 'bathrooms',
    'gość': 'capacity',
    'gości': 'capacity',
    'sypialnia': 'bedrooms',
    'sypialnie': 'bedrooms',
    'sypialni': 'bedrooms',
    'łóżko': 'beds',
    'łóżka': 'beds',
    'łóżek': 'beds',
}

def parse_offer(offer, checkin):
    offer_id = offer.select("a")[0]['target'].split("listing_")[-1]
    offer_price = float(''.join(offer.select("span._tyxjp1")[0].text.split()[:-1]))
    offer_type = " ".join(offer.select("div.mj1p6c8")[0].text.split("w:")[0].split())
    offer_features = [feature.text for feature in offer.select("span.mp2hv9t")]

    current_offer = {
        "id": offer_id,
        "price": offer_price,
        "checkin": checkin,
        "type": offer_type,
    }
    feature_sets = offer.select("div.i1wgresd")
    basic_features = [t.text.lower() for t in feature_sets[0].select("span.mp2hv9t")]
    other_features = [t.text.lower() for t in feature_sets[1].select("span.mp2hv9t")] if len(feature_sets) > 1 else []

    for feature in basic_features:
        split = feature.split()
        if len(split) > 1:
            current_offer[key_map[' '.join(split[1:])]] = split[0]
        else:
            if split[0].lower() == 'studio':
                current_offer['bedrooms'] = 0

    current_offer['wi-fi'] = 'wi-fi' in other_features
    current_offer['kitchen'] = 'kuchnia' in other_features
    current_offer['washing machine'] = 'pralka' in other_features
    current_offer['self check in'] = 'samodzielne zameldowanie' in other_features
    return current_offer

### Przetwarzanie danej daty
Dla każdego dnia w ustalonym przedziale dat przetwarzamy go poprzez pobranie wszystkich 15 stron rezultatów. Dla każdej strony wyciągamy wszystkie elementy zawierające informacje o ofertach, parsujemy je (używając zdefiniowanej powyżej funkcji `parse_offer`) i dodajemy wynikowy słownik do kolekcji `offers`.

In [ ]:
def process_date(checkin_date, num_guests=1):
    checkin = checkin_date.strftime("%Y-%m-%d")
    checkout = (checkin_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    offers = []
    total_num_pages = 15
    page = 1
    end = False
    while page <= total_num_pages:
        items_offset = (page - 1) * 20
        url = f"https://www.airbnb.pl/s/{query}/homes?checkin={checkin}&checkout={checkout}&adults={num_guests}&items_offset={items_offset}"
        url = quote_plus(url)
        response = requests.get(f"{url_prefix}{url}")
        soup = BeautifulSoup(response.content, "html.parser")
        
        for offer in soup.select("div.cm4lcvy"):
            current_offer = parse_offer(offer, checkin=checkin)
            offers.append(current_offer)
        
        page += 1
    return offers

### Proces scrapowania
Teraz po prostu iterujemy po żądanym zakresie dat i przetwarzamy każdy dzień zgodnie z logiką zdefiniowaną w funkcjach powyżej. Zajmie to kilka minut, zależnie od zdefiniowanego zakresu dat.

In [ ]:
offers = []
for date in tqdm(pd.date_range(start_date, end_date, freq=freq)):
    offers += process_date(checkin_date=date)

## Eksploracja danych

Mając dane w pamięci, stwórzmy data frame Pandas i dokonajmy eksploracji. Przy okazji zapiszemy nasze dane do pliku csv i wczytamy jego zawartość, co ma dodatkowy efekt w postaci automatycznej konwersji danych numerycznych na odpowiednie typy.

In [ ]:
offers_df = pd.DataFrame(offers)
offers_df.to_csv('./data-pl.csv', sep=';', index=False)
offers_df = pd.read_csv("./data-pl.csv", sep=";")

Oczyścimy najpierw dane z duplikatów zliczymy je i zliczymy różne typy ofert. Dodatkowo, w ofertach w których jest "współdzielona" łazienka zamienimy ilość na 1.

In [ ]:
offers_df = offers_df.drop_duplicates(subset=['id'], keep='last')
print("Number of data points:", len(offers_df))
print(offers_df['type'].value_counts())
offers_df["bathrooms"].replace({"współdzielona": 1}, inplace=True)

Jak widać jest wiele różnych szczegółowych typów, ale nas bardziej interesują szersze rodzaje stosowane przez Airbnb, tj.: "Całe miejsce", "Pokój prywatny", "Pokój w hotelu" i "Pokój współdzielony". W tym celu dokonamy mapowania szczegółowych typów na ogołne:

In [ ]:
offers_df["type"].replace({
    "Cały obiekt – apartament": "Całe miejsce",
    "Cały apartament z obsługą": "Całe miejsce",
    "Pokój w hostelu": "Pokój prywatny",
    "Cały obiekt – apartament gościnny": "Całe miejsce",
    "Cały obiekt – condo": "Całe miejsce",
    "Pokój w apartamencie z obsługą": "Pokój prywatny",
    "Cały obiekt – loft": "Całe miejsce",
    "Łóżka w hostelu": "Pokój współdzielony",
    "Pokój w hotelu butikowym": "Pokój hotelowy",
    "Cały obiekt – dom": "Całe miejsce",
    "Cały obiekt – domek parterowy": "Całe miejsce",
    "Cały obiekt – domek gościnny": "Całe miejsce",
    "Cały obiekt – chatka": "Całe miejsce",
}, inplace=True)

### Jakie rodzaje ofert są najczęstsze? Ile łóżek spodziewać się dla danego typu?

Możemy teraz narysować histogram, którzy wskaże nam ile znaleźliśmy ofert tych podstawowych typów i przy okazji sprawdźmy ile zazwyczaj łóżek oferowane jest w danym typie:

In [ ]:
plt.figure(figsize=(14,8))
ax = sns.histplot(data=offers_df, x="type", shrink=0.9, alpha=0.9, multiple="stack", hue="beds")
for label in ax.get_xticklabels():
    label.set_rotation(35)
ax.set(xlabel="Rodzaj miejsca", ylabel="Liczba")

Całe miejsca wyraźne dominują i jednocześnie widać, że większość oferuje jedno lub dwa łóżka.

### Cena a maksymalna liczba gości i bardzo drogie oferty

Sprawdzimy teraz jak ceny mają się do maksymalnej liczby gości w oferowanych miejscach. Prawie wszystkie oferty w naszym przypadku mieszczą się w cenie poniżej 2.500 zł, więc wyświetlimy tylko taki przedział:

In [ ]:
plt.figure(figsize=(14,8))
ax = sns.stripplot(x="capacity", y="price", hue="type", dodge=True, data=offers_df)
ax.set(ylim=[0, 2500], xlabel="Maksymalna liczba gości", ylabel="Cena")
ax.legend().set_title("Rodzaj miejsca")

### Najdroższe oferty
Poniżej pobieramy oferty droższe niż 2.500 zł i generujemy linki do nich:

In [ ]:
offers_df[offers_df.price > 2500]

In [ ]:
for offer_id in offers_df[offers_df.price > 1500].id:
    print(f"https://airbnb.pl/rooms/{offer_id}")

## Podsumowanie

Airbnb jest przykładem strony, która nie wyświetla prawie wcale wartościowej treści bez włączonego javascriptu. Funkcjonalność Scraping Fish [renderowania javascriptu](https://scrapingfish.pl/docs/api-options) sprawia, że poradzenie sobie z tym problemem to bułka z masłem.

Ledwie dotknęliśmy zagadnienia eksploracji naszych danych. Żeby zachować prostotę tego posta ograniczyliśmy nasze analizy do stosunkowo małego zbioru danych, żeby zaprezentować co jest możliwe oraz jak łatwo można scrapować strony mocno oparte o javascript. Planujemy jednak pobrać o wiele więcej danych i przeprowadzić znacznie bardziej wnikliwe analizy, więc bądźcie czujni i sprawdzajcie naszego bloga!

Jeśli też scrapujesz dane i budujesz przy użyciu tej technologii produkty sprawdź API Scraping Fish. Możesz [zacząć](https:/scrapingfish.pl/buy) już za 8 zł + VAT bez żadnych zobowiązań.